In [1]:
import select
import psycopg2
import psycopg2.extensions


In [ ]:
from flask import Flask, render_template
from flask_sockets import Sockets
from json import dumps
import gevent
import time
from gevent.pool import Pool

col_names = ['id_str', 'categories', 'in_reply_to_user_id', 'sentiment', 'text', 'created_at',
            'user_location','in_reply_to_status_id','in_reply_to_screen_name',
            'retweet_count','favorite_count','user_name','user_screen_name','user_id_str']

app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            n = len(notify)
            curs.execute("select * from stream order by id_str desc limit %s;"%n)
            rec = curs.fetchall()
            for reci in rec:
                rec_json = dumps(dict(zip(col_names,reci)))
                ws.send(rec_json)
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()
    
@app.route('/')
def homepage():
    try:
        return render_template("index.html")
    except Exception, e:
        return str(e)

if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8090), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()